In [3]:
import tweepy
import pandas as pd
import time

auth = tweepy.OAuthHandler('BcWEliOxF0h6v9VWpUlNtS5nU', 'HfW6WHnnPqB5BkodYKgMTAihGbmEjioJpP3DhAekLLtVeb6xIi')
auth.set_access_token('841021709209731073-rRR8mlOkIw7pLGEnplp03nWwnYOGwJ3', 'Aa2c8dlmUOKzsRRRc0v6gELtKbJFAkcNwNzOe9T6uuCUQ')

api = tweepy.API(auth, wait_on_rate_limit=True) #

# Collect tweets
query = "#chatgpt" + " -filter:retweets"
# cutoff_date = "2023-03-05"
tweets = tweepy.Cursor(api.search_tweets, q=query, lang="en", count=100).items(2000)

tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]

tweets_df = pd.DataFrame(data=tweets_list, columns=['date', 'user', 'location', 'text'])

# A good idea to save downloaded tweets as CSV
tweets_df.to_csv ('./resources/chatgpt.csv', quotechar='"', encoding='utf8', index = False, header=True)
print("Finish data generation.")

Rate limit reached. Sleeping for: 603


Finish data generation.
